# Check environment.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.keras.__version__)

2.1.0
2.2.4-tf


In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


# 1.Read the train and test data.

In [2]:
trainWordsDf = pd.read_csv('conv_train.csv')
testWordsDf = pd.read_csv('conv_test.csv')

# 2.Using Tokenizer result to vectorize the datasets.

Make the vectorization result same length.

In [3]:
import pickle
from tensorflow.keras.preprocessing import sequence
max_len = 300
with open('tok.pickle', 'rb') as handle:
    tok = pickle.load(handle)
train_seq = tok.texts_to_sequences(trainWordsDf.Cut)
test_seq = tok.texts_to_sequences(testWordsDf.Cut)
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
test_seq_mat = sequence.pad_sequences(test_seq,maxlen=max_len)

In [4]:
train_y = trainWordsDf.Flag.values

# 3.Define a model and train it.

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


# inputs = Input(name='inputs',shape=[max_len])
# ## Embedding(size of word table,size of batch,length of every news)
# layer = Embedding(max_words+1,128,input_length=max_len)(inputs)
# layer = LSTM(128)(layer)
# layer = Dense(128,activation="relu",name="FC1")(layer)
# layer = Dropout(0.5)(layer)
# layer = Dense(1,activation="sigmoid",name="FC2")(layer)
# model = Model(inputs=inputs,outputs=layer)
# model.summary()
# model.compile(loss='binary_crossentropy',
#              optimizer='adam',
#              metrics=['accuracy'])

inputs = Input(name='inputs',shape=[max_len])
## Embedding(size of word table,size of batch,length of every news)
layer = Embedding(max_words+1,128,input_length=max_len)(inputs)
layer = Bidirectional(LSTM(128))(layer)
layer = Dense(128,activation="relu",name="FC1")(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,activation="sigmoid",name="FC2")(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 300)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 300, 128)          1280128   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               263168    
_________________________________________________________________
FC1 (Dense)                  (None, 128)               32896     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
FC2 (Dense)                  (None, 1)                 129       
Total params: 1,576,321
Trainable params: 1,576,321
Non-trainable params: 0
___________________________________________________

In [7]:
model_fit = model.fit(train_seq_mat,train_y,batch_size=256,epochs=50)

Train on 135656 samples
Epoch 1/20
135656/135656 [==============================] - 56s 410us/sample - loss: 0.0441 - accuracy: 0.9906
Epoch 2/20
135656/135656 [==============================] - 53s 389us/sample - loss: 0.0317 - accuracy: 0.9913
Epoch 3/20
135656/135656 [==============================] - 54s 396us/sample - loss: 0.0280 - accuracy: 0.9919
Epoch 4/20
135656/135656 [==============================] - 54s 398us/sample - loss: 0.0232 - accuracy: 0.9931
Epoch 5/20
135656/135656 [==============================] - 54s 396us/sample - loss: 0.0180 - accuracy: 0.9945
Epoch 6/20
135656/135656 [==============================] - 53s 394us/sample - loss: 0.0141 - accuracy: 0.9960
Epoch 7/20
135656/135656 [==============================] - 54s 401us/sample - loss: 0.0111 - accuracy: 0.9968
Epoch 8/20
135656/135656 [==============================] - 54s 399us/sample - loss: 0.0090 - accuracy: 0.9976
Epoch 9/20
135656/135656 [==============================] - 53s 392us/sample - loss: 0.0

# 4.Store the train history, model, perdect the test data and store the result.

In [8]:
f = open('history.txt','w')
f.write(str(model_fit.history))
f.close()

In [9]:
## Model`s save and download.
from tensorflow.keras.models import load_model
# Save
model.save('my_model.h5')  
del model  # deletes the existing model
# Downlod
model = load_model('my_model.h5')

In [10]:
predict = model.predict(test_seq_mat)

In [11]:
resultDf = pd.DataFrame(range(1,(len(testWordsDf.index)+1)),columns=['SessionId'])
resultDf['Probability'] = predict
resultDf.to_csv('result.csv', index=0)
resultDf

,SessionId,Probability
0,1,7.053516e-10
1,2,8.285665e-11
2,3,3.003254e-09
3,4,2.452652e-05
4,5,5.076798e-09
...,...,...
50064,50065,2.853299e-04
50065,50066,1.148497e-03
50066,50067,1.152184e-05
50067,50068,9.443950e-08
